In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [34]:
import xgboost as xgb
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
import numpy as np
np.set_printoptions(threshold=np.inf)
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans,DBSCAN
from sklearn.cluster import AgglomerativeClustering as ac
from sklearn.linear_model import LogisticRegression as LR
from sklearn.model import train_test_split as tts
from sklearn.metrics import accuracy_score as acc
from sklearn import SVC
from sklearn import metrics
from sklearn.metrics import  plot_roc_curve
from sklearn.model_selection import  train_test_split
import getdata.FlightDelay as fd
import getdata.XgboostModel as xm
from sklearn.model_selection import train_test_split


In [26]:
# ——————————读取数据——————————#
# 航班数据
flight_data_2009 = pd.read_csv('./source/2009.csv',encoding='unicode_escape')
# flight_data_2010 = pd.read_csv('./source/2010.csv',encoding='unicode_escape')
# flight_data_09_10 = pd.concat(
   # [flight_data_2009,flight_data_2010]).reset_index(drop=True)
#flight_data_2011 = pd.read_csv('./source/2011.csv',encoding='unicode_escape')
#flight_data_2012 = pd.read_csv('./source/2012.csv',encoding='unicode_escape')
#flight_data_2013 = pd.read_csv('./source/2013.csv',encoding='unicode_escape')
#flight_data_09_13 = pd.concat(
#    [flight_data_2009,flight_data_2010,flight_data_2011,flight_data_2012,flight_data_2013]).reset_index(drop=True)

#flight_data_2014 = pd.read_csv('./source/2014.csv',encoding='unicode_escape')
#flight_data_2015 = pd.read_csv('./source/2015.csv',encoding='unicode_escape')
#flight_data_2016 = pd.read_csv('./source/2016.csv',encoding='unicode_escape')
#flight_data_2017 = pd.read_csv('./source/2017.csv',encoding='unicode_escape')
#flight_data_2018 = pd.read_csv('./source/2018.csv',encoding='unicode_escape')

In [27]:
# 保存数据集
flight_data_2009.to_csv('./clear_data/temp_train_data.csv',index=False)
flight_data_2009

,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,...,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 27
0,2009-01-01,XE,1204,DCA,EWR,1100,1058.0,-2.0,18.0,1116.0,...,62.0,68.0,42.0,199.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2009-01-01,XE,1206,EWR,IAD,1510,1509.0,-1.0,28.0,1537.0,...,82.0,75.0,43.0,213.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2009-01-01,XE,1207,EWR,DCA,1100,1059.0,-1.0,20.0,1119.0,...,70.0,62.0,36.0,199.0,NaN,NaN,NaN,NaN,NaN,NaN
3,2009-01-01,XE,1208,DCA,EWR,1240,1249.0,9.0,10.0,1259.0,...,77.0,56.0,37.0,199.0,NaN,NaN,NaN,NaN,NaN,NaN
4,2009-01-01,XE,1209,IAD,EWR,1715,1705.0,-10.0,24.0,1729.0,...,105.0,77.0,40.0,213.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6429333,2009-12-31,YV,2846,YUM,PHX,615,605.0,-10.0,19.0,624.0,...,67.0,71.0,39.0,160.0,NaN,NaN,NaN,NaN,NaN,NaN
6429334,2009-12-31,YV,2850,YUM,PHX,1929,1919.0,-10.0,7.0,1926.0,...,70.0,61.0,48.0,160.0,NaN,NaN,NaN,NaN,NaN,NaN
6429335,2009-12-31,YV,2866,YUM,PHX,1400,1400.0,0.0,29.0,1429.0,...,55.0,70.0,32.0,160.0,0.0,0.0,15.0,0.0,0.0,NaN
6429336,2009-12-31,YV,2868,YUM,PHX,1620,1615.0,-5.0,10.0,1625.0,...,67.0,61.0,45.0,160.0,NaN,NaN,NaN,NaN,NaN,NaN


数据集列的含义
FL_DATE 航班日期，yy/mm/dd//
OP_CARRIER 航空公司标识符//
OP_CARRIER_FL_NUM 航班号//
ORIGIN 起始机场代码//
DEST 目的地机场代码//
CRS_DEP_TIME 计划出发时间//
DEP_TIME 实际出发时间//
DEP_DELAY 出发总延误分钟//
TAXI_OUT 从始发机场登机口出发到飞机起飞之间经过的时间，起飞滑行时间
WHEELS_OFF 飞机轮子离开地面的时间点
WHEELS_ON 飞机轮子接触地面的时间点
TAXI_IN 从机轮启动到到达目的地机场登机口所经过的时间，降落滑行时间
CRS_ARR_TIME 计划到达时间//
ARR_TIME 实际到达时间//
ARR_DELAY 到达总延误分钟//
CANCELLED 航班取消（1 = 取消）//
CANCELLATION_CODE 取消代码 A - 航空公司/承运人；B - 天气；C - 国家航空系统；D - 安全
DIVERTED 飞机降落在不合时宜的机场
CRS_ELAPSED_TIME 飞行旅行所需的计划时间量//
ACTUAL_ELAPSED_TIME AIR_TIME+TAXI_IN+TAXI_OUT
AIR_TIME wheels_off 和 wheels_on 时间之间的持续时间
DISTANCE 两个机场之间的距离
CARRIER_DELAY 几分钟内航空公司造成的延误
WEATHER_DELAY 天气原因延误//
NAS_DELAY 空气系统造成的延迟
SECURITY_DELAY 安全问题造成的延误
LATE_AIRCRAFT_DELAY 安检造成的延误

1.计划飞行时间（单位：分钟）【间接反映航线距离】
2.计划起飞日期&计划到达日期【用以匹配天气信息】yyyy-mm-dd
3.起飞延误时间（单位：分钟）负数为提前，正数为延误
4.到达延误时间（单位：分钟）
5.天气用机场和时间去连接

In [28]:
# 读取训练集
flight_data = pd.read_csv('./clear_data/temp_train_data.csv')

# 重命名列
flight_data['航班是否取消'] = flight_data['CANCELLED']

flight_data['航班日期'] = flight_data['FL_DATE']
flight_data['航空公司'] = flight_data['OP_CARRIER']
flight_data['航班号'] = flight_data['OP_CARRIER_FL_NUM']
flight_data['出发机场'] = flight_data['ORIGIN']
flight_data['到达机场'] = flight_data['DEST']

flight_data['计划起飞时间'] = flight_data['CRS_DEP_TIME']
flight_data['实际起飞时间'] = flight_data['DEP_TIME']
flight_data['出发延误总分钟'] = flight_data['DEP_DELAY']

flight_data['计划到达时间'] = flight_data['CRS_ARR_TIME']
flight_data['实际到达时间'] = flight_data['ARR_TIME']
flight_data['到达延误总分钟'] = flight_data['ARR_DELAY']

# 计划飞行时间
flight_data['计划飞行时间'] = flight_data['计划到达时间'] - flight_data['计划起飞时间']

In [29]:
# 删除用不到的字段
del flight_data['FL_DATE']
del flight_data['OP_CARRIER']
del flight_data['CANCELLED']
del flight_data['OP_CARRIER_FL_NUM']
del flight_data['ORIGIN']
del flight_data['DEST']
del flight_data['CRS_DEP_TIME']
del flight_data['DEP_TIME']
del flight_data['DEP_DELAY']
del flight_data['CRS_ARR_TIME']
del flight_data['ARR_TIME']
del flight_data['ARR_DELAY']
del flight_data['LATE_AIRCRAFT_DELAY']
del flight_data['SECURITY_DELAY']
del flight_data['NAS_DELAY']
del flight_data['CARRIER_DELAY']
del flight_data['DISTANCE']
del flight_data['AIR_TIME']
del flight_data['ACTUAL_ELAPSED_TIME']
del flight_data['DIVERTED']
del flight_data['CANCELLATION_CODE']
del flight_data['TAXI_IN']
del flight_data['WHEELS_ON']
del flight_data['WHEELS_OFF']
del flight_data['TAXI_OUT']
del flight_data['Unnamed: 27']
del flight_data['CRS_ELAPSED_TIME']
del flight_data['WEATHER_DELAY']

In [30]:
# 实际到达时间超过计划到达时间15min或者取消航班作为延误的判断指标，给数据打上标签
def isDelay(t,c):
    if(t >= 15 or c==1): return 1
    else: return 0

flight_data['航班是否延误'] = flight_data.apply(
    lambda row: isDelay(row['到达延误总分钟'],row['航班是否取消']),axis=1)

In [31]:
# 判断正负样本的分布
flight_data['航班是否延误'].value_counts()

航班是否延误
0    5171799
1    1257539
Name: count, dtype: int64

In [37]:
# import missingno as msn
# msn.matrix(flight_data)

ModuleNotFoundError: No module named 'missingno'

In [32]:
# 看看是否有缺失值
flight_data.isnull().any()

航班是否取消     False
航班日期       False
航空公司       False
航班号        False
出发机场       False
到达机场       False
计划起飞时间     False
实际起飞时间      True
出发延误总分钟     True
计划到达时间     False
实际到达时间      True
到达延误总分钟     True
计划飞行时间     False
航班是否延误     False
dtype: bool

In [33]:
# 缺失值处理
flight_data['实际起飞时间'] = flight_data['实际起飞时间'].fillna(flight_data['实际起飞时间'].mean()) #均值填充
flight_data['出发延误总分钟'] = flight_data['出发延误总分钟'].fillna(flight_data['出发延误总分钟'].mean()) #均值填充
flight_data['实际到达时间'] = flight_data['实际到达时间'].fillna(flight_data['实际到达时间'].mean()) #均值填充
flight_data['到达延误总分钟'] = flight_data['到达延误总分钟'].fillna(flight_data['到达延误总分钟'].mean()) #均值填充
flight_data['出发延误总分钟'] = flight_data['出发延误总分钟'].fillna(flight_data['出发延误总分钟'].mean()) #均值填充

In [34]:
flight_data

,航班是否取消,航班日期,航空公司,航班号,出发机场,到达机场,计划起飞时间,实际起飞时间,出发延误总分钟,计划到达时间,实际到达时间,到达延误总分钟,计划飞行时间,航班是否延误
0,0.0,2009-01-01,XE,1204,DCA,EWR,1100,1058.0,-2.0,1202,1206.0,4.0,102,0
1,0.0,2009-01-01,XE,1206,EWR,IAD,1510,1509.0,-1.0,1632,1624.0,-8.0,122,0
2,0.0,2009-01-01,XE,1207,EWR,DCA,1100,1059.0,-1.0,1210,1201.0,-9.0,110,0
3,0.0,2009-01-01,XE,1208,DCA,EWR,1240,1249.0,9.0,1357,1345.0,-12.0,117,0
4,0.0,2009-01-01,XE,1209,IAD,EWR,1715,1705.0,-10.0,1900,1822.0,-38.0,185,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6429333,0.0,2009-12-31,YV,2846,YUM,PHX,615,605.0,-10.0,722,716.0,-6.0,107,0
6429334,0.0,2009-12-31,YV,2850,YUM,PHX,1929,1919.0,-10.0,2039,2020.0,-19.0,110,0
6429335,0.0,2009-12-31,YV,2866,YUM,PHX,1400,1400.0,0.0,1455,1510.0,15.0,55,1
6429336,0.0,2009-12-31,YV,2868,YUM,PHX,1620,1615.0,-5.0,1727,1716.0,-11.0,107,0


In [17]:
# 读取机场和对应城市
airport_city = pd.read_csv('./source/us_airports.csv',encoding='unicode_escape',low_memory=False)

In [18]:
# 看看是否有缺失值
airport_city.isnull().any()

id                   False
ident                False
type                 False
name                 False
latitude_deg         False
longitude_deg        False
elevation_ft          True
continent             True
country_name         False
iso_country          False
region_name          False
iso_region           False
local_region         False
municipality          True
scheduled_service    False
gps_code              True
iata_code             True
local_code            True
home_link             True
wikipedia_link        True
keywords              True
score                False
last_updated         False
dtype: bool

In [19]:
# 处理缺失值
airport_city['iata_code'] = airport_city['iata_code'].fillna(
    airport_city['iata_code'].value_counts().index[0]) #众数填充

In [20]:
# 重命名
# colNameDict = {
#     'season':'季节',
#     'holiday':'节假日',
#     'workingday':'工作日',
#     'weather':'天气',
#     'temp':'摄氏温度',
#     'atemp':'体感温度',
#     'humidity':'湿度',
#     'windspeed':'风速',
#     'casual':'非注册用户个数',
#     'registered':'注册用户个数',
#     'count':'租车总人数'
# }
# df.rename(columns = colNameDict,inplace=True)
airport_city['机场代号'] = airport_city['iata_code']
airport_city['地区代号'] = airport_city['local_region']
airport_city['地区名字'] = airport_city['region_name']

In [21]:
# 删除第0行
airport_city = airport_city.drop(airport_city.index[0])

In [22]:
airport_city.columns

Index(['id', 'ident', 'type', 'name', 'latitude_deg', 'longitude_deg',
       'elevation_ft', 'continent', 'country_name', 'iso_country',
       'region_name', 'iso_region', 'local_region', 'municipality',
       'scheduled_service', 'gps_code', 'iata_code', 'local_code', 'home_link',
       'wikipedia_link', 'keywords', 'score', 'last_updated', '机场代号', '地区代号',
       '地区名字'],
      dtype='object')

In [23]:
del airport_city['id']
del airport_city['ident']
del airport_city['type']
del airport_city['name']
del airport_city['latitude_deg']
del airport_city['longitude_deg']
del airport_city['elevation_ft']
del airport_city['continent']
del airport_city['country_name']
del airport_city['iso_country']
del airport_city['region_name']
del airport_city['iso_region']
del airport_city['local_region']
del airport_city['municipality']
del airport_city['scheduled_service']
del airport_city['gps_code']
del airport_city['iata_code']
del airport_city['local_code']
del airport_city['home_link']
del airport_city['wikipedia_link']
del airport_city['keywords']
del airport_city['score']
del airport_city['last_updated']

In [24]:
airport_city.columns

Index(['机场代号', '地区代号', '地区名字'], dtype='object')

In [35]:
flight_data.columns

Index(['航班是否取消', '航班日期', '航空公司', '航班号', '出发机场', '到达机场', '计划起飞时间', '实际起飞时间',
       '出发延误总分钟', '计划到达时间', '实际到达时间', '到达延误总分钟', '计划飞行时间', '航班是否延误'],
      dtype='object')

In [58]:
# 拼接机场和对应城市
flight_city = pd.merge(flight_data, airport_city,
                       left_on=['出发机场'],right_on=['机场代号'],how='left',sort=False)
flight_city.rename(columns={'地区代号':'出发地区代号'},inplace=True)
flight_city.rename(columns={'地区名字':'出发地区名字'},inplace=True)

flight_city = pd.merge(flight_city, airport_city,
                       left_on=['到达机场'],right_on=['机场代号'],how='left',sort=False)
flight_city.rename(columns={'地区代号':'到达地区代号'},inplace=True)
flight_city.rename(columns={'地区名字':'到达地区名字'},inplace=True)

del flight_city['机场代号_x']
del flight_city['机场代号_y']

In [59]:
flight_city = flight_city.dropna(axis=0,how='any')

In [60]:
# flight_city.isnull().any()

航班是否取消     False
航班日期       False
航空公司       False
航班号        False
出发机场       False
到达机场       False
计划起飞时间     False
实际起飞时间     False
出发延误总分钟    False
计划到达时间     False
实际到达时间     False
到达延误总分钟    False
计划飞行时间     False
航班是否延误     False
出发地区代号     False
出发地区名字     False
到达地区代号     False
到达地区名字     False
dtype: bool

In [61]:
flight_city.to_csv('./clear_data/flight_city_data.csv',index=False)

AttributeError: 'function' object has no attribute 'T'

In [ ]:
# 抽取随机样本


In [ ]:
# 做ROC图像
# plt.rcParams['font.sans-serif'] = ['Microsoft YaHei']
# plt.rcParams['axes.unicode_minus'] = False;
#
# fig = plt.figure(figsize=(8,8),dpi = 100,facecolor='white',edgecolor='black')
# ax.fig.add_subplot(1,1,1)
# ax.plot(fpr_CART_Class,c='red')
# ax.plot(fpr_lr,tpr_lr,c='orange')
# plt.legend(('决策树','logistic回归'),loc = 'best')
# plt.show(fig)
# fig.savefig('./figure/roc.jpg')
# plt.close(fig)

In [ ]:
#xgboost

# 创建航班延误数据集对象
class FlightDelay:
    def __init__(self, feature_names, data, target_names, target):
        self.feature_names = feature_names
        self.data = data
        self.target_names = target_names
        self.target = target


In [ ]:
# 读取excel文件(**强烈建议转为csv去处理，否则数据太容易被误改)
def getDataSet(url, sheetname):

    # 读取路径的xlsx文件
    dataFrame = pd.read_excel(url, sheet_name = sheetname)

    # 可以打印查看数据的类型是否有str
    # dataInfo = dataFrame.info()

    # 转为数据矩阵
    data_matrix = dataFrame.values

    # 获取feature_names,取第一行为feature,使用columns获取列名，然后使用values获取结果，去除掉最后的label列名
    feature_names = dataFrame.head(n=0).columns.values[:-1]

    # 获取label标签
    target_names = dataFrame.head(n=0).columns.values[-1]

    # feature特征矩阵data,全部取整，保证离散值足够小
    data = data_matrix[:, :-1].astype(int)

    # label标签向量，将小数转为整型
    target = data_matrix[:, -1].astype(int)

    return FlightDelay(feature_names, data, target_names, target)

# XGBOOST进行预测

# 通用参数：#
# booster：我们有两种参数选择，gbtree和gblinear。gbtree是采用树的结构来运行数据，而gblinear是基于线性模型
# silent：静默模式，为1时模型运行不输出
# nthread: 使用线程数，一般我们设置成-1,使用所有线程。如果有需要，我们设置成多少就是用多少线程

# Booster参数：#
# n_estimator:num_boosting_rounds 最大的迭代次数
# learning_rate: 有时也叫作eta，系统默认值为0.3,每一步迭代的步长，很重要。太大了运行准确率不高，太小了运行速度慢。我们一般使用比默认值小一点，0.1左右就很好
# gamma：系统默认为0,在节点分裂时，只有分裂后损失函数的值下降了，才会分裂这个节点。gamma指定了节点分裂所需的最小损失函数下降值。 这个参数的值越大，算法越保守。因为gamma值越大的时候，损失函数下降更多才可以分裂节点。所以树生成的时候更不容易分裂节点
# subsample：系统默认为1，这个参数控制对于每棵树，随机采样的比例。减小这个参数的值，算法会更加保守，避免过拟合。但是，如果这个值设置得过小，它可能会导致欠拟合。 典型值：0.5-1，0.5代表平均采样，防止过拟合
# colsample_bytree：系统默认值为1。我们一般设置成0.8左右，用来控制每棵随机采样的列数的占比(每一列是一个特征)。 典型值：0.5-1
# colsample_bylevel：默认为1,我们也设置为1；这个就相比于前一个更加细致了，它指的是每棵树每次节点分裂的时候列采样的比例
# max_depth： 系统默认值为6，我们常用3-10之间的数字。这个值为树的最大深度。这个值是用来控制过拟合的。max_depth越大，模型学习的更加具体。设置为0代表没有限制
# max_delta_step：默认0,我们常用0.，这个参数限制了每棵树权重改变的最大步长，如果这个参数的值为0,则意味着没有约束。如果他被赋予了某一个正值，则是这个算法更加保守。通常，这个参数我们不需要设置，但是当个类别的样本极不平衡的时候，这个参数对逻辑回归优化器是很有帮助的。
# lambda:也称reg_lambda,默认值为0就，权重的L2正则化项。(和Ridge regression类似)。这个参数是用来控制XGBoost的正则化部分的。这个参数在减少过拟合上很有帮助。
# alpha:也称reg_alpha默认为0，权重的L1正则化项。(和Lasso regression类似)。 可以应用在很高维度的情况下，使得算法的速度更快
# scale_pos_weight：默认为1，在各类别样本十分不平衡时，把这个参数设定为一个正值，可以使算法更快收敛。通常可以将其设置为负样本的数目与正样本数目的比值

# 学习目标参数 #
# objective [缺省值=reg:linear]
# reg:linear– 线性回归
# reg:logistic – 逻辑回归
# binary:logistic – 二分类逻辑回归，输出为概率
# binary:logitraw – 二分类逻辑回归，输出的结果为wTx
# count:poisson – 计数问题的poisson回归，输出结果为poisson分布。在poisson回归中，max_delta_step的缺省值为0.7
# multi:softmax – 设置 XGBoost 使用softmax目标函数做多分类，需要设置参数num_class（类别个数），输出为概率最大的分类
# multi:softprob – 如同softmax，但是输出结果为ndata*nclass的向量，其中的值是每个数据分为每个类的概率

# eval_metric [缺省值=通过目标函数选择]
# rmse: 均方根误差(回归问题默认)
# mae: 平均绝对值误差
# logloss: negative log-likelihood数似然损失，对数损失函数，一般用于分类问题
# error: 二分类错误率。其值通过错误分类数目与全部分类数目比值得到。对于预测，预测值大于0.5被认为是正类，其它归为负类（分类问题默认）
# merror: 多分类错误率，计算公式为(wrong cases)/(all cases)
# mlogloss: 多分类log损失
# auc: 曲线下的面积
# ndcg: Normalized Discounted Cumulative Gain
# map: 平均正确率

def inputDataSet(X_train, X_test, y_train, y_test):

    # 设置xgboost分类器
    xlf = xgb.XGBClassifier(max_depth=6, learning_rate=0.1, n_estimators=1000, objective='binary:logistic',
                            nthread=-1, subsample=0.5, colsample_bytree=0.8)

    # 训练，verbose标识每迭代一次进行输出，可以指定迭代几次输出
    xlf.fit(X_train, y_train, eval_metric='error', verbose=True)

    # 预测准确率得分
    score = xlf.score(X_test, y_test)
    print("预测准确率得分 = {:.2f}".format(score),)

    # AUC模型评分
    y_pred = xlf.predict_proba(X_test)
    auc_score = roc_auc_score(y_test, y_pred[:, 1])
    print("AUC模型评价 = {:.2f}".format(auc_score))

    # roc曲线可视化输出
    roc_function_image(y_test, y_pred)

# 可视化的roc评价函数图像
def roc_function_image(y_test, y_pred):

    # fpr是假阳性率，tpr是真阳性率
    fpr, tpr, _ = roc_curve(y_test, y_pred[:, 1])

    plt.plot(fpr, tpr, color='darkred')
    plt.plot([0, 1], [0, 1], color='grey', lw=1, linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.show()

# 导入xlsx文件路径和工作簿名称
url = "./source/2009.csv"
sheet_name = 'DL'

# 调用方法获取数据集
flightDataSet = fd.getDataSet(url, sheet_name)
target_names = flightDataSet.target_names
target = flightDataSet.target
feature_names = flightDataSet.feature_names
data = flightDataSet.data

# 使用train_test_split方法获取训练集和测试集，这里二八分，随机种子为1
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=1)

# 使用xgboost进行模型预测
xm.inputDataSet(X_train, X_test, y_train, y_test)